In [19]:
#preprocessing for training pipeline
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

In [20]:
fname_prefix = "/data/users/team2_capstone/code-style-probing/"

target_features = [
     'snake_case_var_ratio',
     'snake_case_class_ratio',
     'snake_case_method_ratio',
     'upper_camel_case_var_ratio',
     'upper_camel_case_class_ratio',
     'upper_camel_case_method_ratio',
     'lower_camel_case_var_ratio',
     'lower_camel_case_class_ratio',
     'lower_camel_case_method_ratio',
     'func_decorators_avg',
     'class_decorators_avg',
     'class_parents_avg',
     'comprehensions_avg',
     'generators_avg',
     'lambda_avg',
     'comment_density',
     'ds_density',
]
PY150K_DIR = fname_prefix + "data/py150"
PY150K_CODE_DIR = fname_prefix + "data/py150/py150_files"
PY150K_TRAIN_AST = fname_prefix + "data/py150/python100k_train.json"
PY150K_EVAL_AST = fname_prefix + "data/py150/python50k_eval.json"
PY150K_TRAIN_CODE = fname_prefix + "data/py150/py150_files/python100k_train.txt"
PY150K_EVAL_CODE = fname_prefix + "data/py150/py150_files/python50k_eval.txt"

In [21]:
combined_df = pd.read_csv(fname_prefix+"data/bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file0_20220530.csv")
#os.listdir("data")
bq_metric_files = [
    #'bigquery_metric_dedup_file0_20220530.csv',
    #'py150k_metric_20220527.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file1_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file2_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file3_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file4_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file5_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file6_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file7_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file8_20220530.csv',
    'bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file9_20220530.csv' 
 ]

for file in bq_metric_files:
    temp_df = pd.read_csv(f'{fname_prefix}data/{file}')
    combined_df = pd.concat((combined_df, temp_df))
#os.listdir(f'{fname_prefix}data/BigQuery/files/')

#need to load all of the data

bq_content_df = pd.read_csv(f'{fname_prefix}data/BigQuery/files/cubert_metadata000000000000') 
bq_content_files = [
    f'data/BigQuery/files/cubert_metadata000000000001',
    f'data/BigQuery/files/cubert_metadata000000000002',
    f'data/BigQuery/files/cubert_metadata000000000003',
    f'data/BigQuery/files/cubert_metadata000000000004',
    f'data/BigQuery/files/cubert_metadata000000000005',
    f'data/BigQuery/files/cubert_metadata000000000006',
    f'data/BigQuery/files/cubert_metadata000000000007',
    f'data/BigQuery/files/cubert_metadata000000000008',
    f'data/BigQuery/files/cubert_metadata000000000009'
]
for file in bq_content_files:
    temp_df = pd.read_csv(f'{fname_prefix}{file}')
    bq_content_df = pd.concat((bq_content_df, temp_df))
display(combined_df)
#combined_df = pd.concat([py150k_df, bigquery_df], axis = 0) 
combined_df['file'] = [ x + y  for x, y in zip(combined_df['repository'], combined_df['filepath'])]
bq_content_df['file'] = [ x + y  for x, y in zip(bq_content_df['repository'], bq_content_df['filepath'])]
print (len (bq_content_df['file']))

,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,comment_avg_len,comment_density,...,other_case_class,func_async_count,generators,lower_case_class,snake_case_class,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method
0,jbkalmbach/kbmod,analysis/rawGenerated/batchGenerate.py,1,32,1,1,51.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pymedusa/SickRage,medusa/providers/torrent/rss/rsstorrent.py,22,763,27,1,170.0,8.0,17.625000,0.047059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ipdata/python,ipdata/test_cli.py,6,15,16,1,135.0,34.0,37.352941,0.251852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,26.818182,0.186441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hidat/audio_pipeline,review_parser/mb_release.py,1,26,4,1,76.0,12.0,37.166667,0.157895,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87927,operepo/ope,libs/gluon/packages/dal/pydal/adapters/sqlite.py,8,138,9,1,279.0,14.0,43.857143,0.050179,...,2.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,2.0
87928,DOV-Vlaanderen/pydov,setup.py,17,287,31,1,66.0,3.0,25.666667,0.045455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87929,slogan621/tscharts,apps/xrayuploader/xrayuploader.py,8,97,16,1,502.0,20.0,49.300000,0.039841,...,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
87930,seasonstar/bibi,application/models/order/logistic.py,365,12,1016,1,434.0,1.0,23.000000,0.002304,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1269088


In [22]:
#!pip install hdbscan
import hdbscan
print (os.listdir(fname_prefix + "data/combined_dataset/clusters/all_data"))
with open(fname_prefix + "data/combined_dataset/clusters/all_data/full_feature_clusterer_preds.pickle", "rb") as file:
    cluster_pred = pickle.load(file)
    labels = cluster_pred.labels_
    cluster_num = len(np.unique(labels))
combined_df['labels'] = labels.tolist()[ : 87273] + labels.tolist()[187273: ] 

['full_feature_clusterer_preds.pickle', 'feature_list_params.txt']


In [23]:
#get code


code_filenames = read_py150k_code(PY150K_TRAIN_CODE)
#print ((code_filenames[1]))
py150_code = []
"""for i in range(0, len(code_filenames)):
    #print (i)
    try:
        py150_code.append( read_file_to_string( #regex codefilenames drop data/py150/py150_files/data/ 
            f"{PY150K_CODE_DIR}/{code_filenames[i]}"
        ))
    except: # currently appending empty string for empy files
        print (f"{PY150K_CODE_DIR}/{code_filenames[i]}")
        py150_code.append( "File Error"
        )""" # getting py 150 code
ex_files = list(combined_df['file'])
filtered_bq = bq_content_df[bq_content_df['file'].isin(ex_files)]
bigquery_code = list(filtered_bq['content'])

In [18]:
#hugging face dataset

print (len(filtered_bq))

combined_code =bigquery_code[ : 87273] + py150_code + bigquery_code[87273 : ] # changing to fit format of bq manifest 1 py150 then rest of bq, done to match cluster
#87273
test_df = pd.read_csv(fname_prefix+"data/bigquery_metric_dedup_file_20220530/bigquery_metric_dedup_file0_20220530.csv")
#display (test_df)
combined_df['content'] = combined_code
display(combined_df)
combined_df.to_csv(fname_prefix + 'data/labeled_code/bq_data_outliers.csv')
#print (combined_df.iloc[87273])
#print (test_df.iloc[87273])

875096


ValueError: Length of values (875096) does not match length of index (208311)

In [15]:
#df to dataset
combined_df = combined_df[combined_df['labels'] != -1]
display(combined_df)
combined_df.to_csv(fname_prefix + 'data/labeled_code/bq_data_no_outliers.csv')
dataset = Dataset.from_pandas(combined_df).train_test_split(test_size=0.2)
"""style_chars = []
for i in range (-1, 25):
    style_chars.append(f"<style{i}>")
print (style_chars)"""

,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,comment_avg_len,comment_density,...,lower_case_class,snake_case_class,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content
8,osrg/gobgp,test/scenario_test/bgp_zebra_test.py,687,1066,2972,1,312.0,40.0,37.925000,0.128205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,osrg/gobgptest/scenario_test/bgp_zebra_test.py,7,# Copyright (C) 2015 Nippon Telegraph and Tele...
12,CivicKnowledge/ambry,test/bundle_tests/ingest.example.com/headersty...,8,303,4,1,6.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CivicKnowledge/ambrytest/bundle_tests/ingest.e...,20,import ambry.bundle \n\n\nclass Bundle(ambry.b...
16,Sistemas-Multimedia/Icecast-tracker,Django/rocio333/holamundo333/manage.py,6,3,2,1,11.0,1.0,21.000000,0.090909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sistemas-Multimedia/Icecast-trackerDjango/roci...,13,#!/usr/bin/env python\nimport os\nimport sys\n...
18,neutrons/FastGR,addie/processing/mantid/master_table/table_row...,3,136,1,1,1014.0,69.0,27.811594,0.068047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutrons/FastGRaddie/processing/mantid/master_...,6,"from __future__ import (absolute_import, divis..."
25,JuBra/GEMEditor,GEMEditor/map/ui/__init__.py,2,39,1,1,3.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JuBra/GEMEditorGEMEditor/map/ui/__init__.py,17,from .MapListDialog import Ui_MapListDialog\nf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87912,limafabio/lazycf,tests/Test_LazyCF.py,1,4,1,1,55.0,1.0,13.000000,0.018182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,limafabio/lazycftests/Test_LazyCF.py,7,#!/usr/bin/py\n\nimport os\nimport sys\nimport...
87917,QQuick/Transcrypt,transcrypt/docs/sphinx/conf.py,231,922,2175,1,251.0,148.0,46.972973,0.589641,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QQuick/Transcrypttranscrypt/docs/sphinx/conf.py,10,import sys\r\n\r\nsys.path.append ('../../modu...
87920,kytulendu/Gw2Browser,extern/glew/build/conan/test_package/conanfile.py,16,5,43,1,22.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kytulendu/Gw2Browserextern/glew/build/conan/te...,20,"from conans import ConanFile, CMake\nimport os..."
87923,martinstastny/django-simplestore,simplestore/products/migrations/0003_auto_2017...,11,1,36,1,25.0,2.0,35.500000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,martinstastny/django-simplestoresimplestore/pr...,20,# -*- coding: utf-8 -*-\n# Generated by Django...


'style_chars = []\nfor i in range (-1, 25):\n    style_chars.append(f"<style{i}>")\nprint (style_chars)'

In [16]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, PLBartTokenizer

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

#tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="python" )
#return_tensors ='pt'


loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/5941df5e4315c5ab63

In [17]:

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
def tokenization(example):
    #print (tokenizer(example["content"], padding='max_length', truncation=True).keys())
    return tokenizer(example["content"], padding='max_length', truncation=True)
train_dataset = dataset["train"].map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.format['type']

test_dataset = dataset["test"].map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.format['type']



loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/5941df5e4315c5ab63

  0%|          | 0/167 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [ ]:
print ((train_dataset[0]))
train_dataset.save_to_disk(fname_prefix + "datasets/codet5_train_full.hf")
test_dataset.save_to_disk(fname_prefix + "datasets/codet5_test_full.hf")
#print (combined_df[combined_df['content'] ==  "File Error"])
combined_df.to_csv(fname_prefix + 'data/labeled_code/full_data.csv')
#"/data/users/team2_capstone/code-style-probing/"

{'labels': tensor(20), 'input_ids': tensor([    1,  8395,  6289,  1953,   364,   450, 22536,  8395,   203,   203,
         2080,  2792,  1930,  6057,   203,   203,   203,  1106,  3877,   559,
           12,  3572,  4672,   203,   565,  1983,  2881, 19064,   273,   296,
         2854,  2881, 19064,    11,   203,   565,  1339,   273,   296,  3184,
           11,   203,   565, 23720,   273,   296, 19064,    11,   203,   565,
         1525, 19064,   273,   296,  5748, 19064,    11,   203,   565,  9549,
         1385,   273,   296, 19525,  1385,    11,   203,   565,   498,  7652,
          273,   296,   856,  7652,    11,   203,   203,   203,  3184,   723,
           67, 30288,   273,   288,   203,   565,  3877,   559,    18,  2854,
         2881, 19064,    30,   296,    37,  1983,  2548,    16,   675,    17,
          591,  1134, 23720,  1093,    16,   203,   565,  3877,   559,    18,
         3184,    30,   296,    37,  8566,  8044, 22525,  1339,   598, 13494,
         1093,    16,   203,

In [ ]:
from datasets import load_from_disk
#example method for loading datset checkpoints
test = load_from_disk(fname_prefix + 'datasets/codet5_test_full.hf')
test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
print ((test[0]))

{'labels': tensor(17), 'input_ids': tensor([    1,  5666,  1062,    18,  2625,   203,   203,  4646,   273,  1062,
           18,  2625,    18,  3693,  1952,  2932, 22851,  4757,    18,   832,
         7923,   203,   203,  7648,   273,   315, 13465,  1611, 23899, 15168,
         2499,  2787,  1611, 15168, 11664,    64,    86,    64,    82,  1350,
           17, 15087,    30,   646,    17,   892,    31,   508,  5189, 11351,
        31670,    86,    64,    82,    64,    86,    64,    82,  3215,    64,
           86,    64,    82, 13465,  1611, 23899, 15168,  2499,  2787,  1611,
        15168, 11664,   413,    64,    86,    64,    82,     6,   203,   203,
         2485,   273,   288,   296,  1350,    17,   559,  4278,   315, 16404,
           19,   687,    17,   892,    31,  7679,    33,  6062,  1611, 23899,
        15168,  2499,  2787,  1611, 15168, 11664,     6,   289,   203,   203,
         4646,    18,  2293,  2932,  3798,  3113,  2206, 30250,  3113,  2385,
           16,  1607,    13,